In [1]:
from itertools import permutations
import numpy as np
import requests
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from tqdm.notebook import tqdm
import pandas as pd
from pprint import pprint
# URL to make requests: fill in x and y coordinates
URL = 'http://65.108.84.176:8080/{}/{}'

# Task

### Solve the testing environment with linear regression.

Objectives:
 1. Grid sample the environment.
 2. Fit the linear regression model.
 3. If the linear function describes the space, the environment is linearly deterministic:
    * Verify the linearity of data.
    * Solve the deterministic environment knowing the linear function. 
    * Build a path through the 10 best points using brute force.

In [2]:
def make_api_call(x: float = 0, y: float = 0) -> float:
    """Make API call to the testing server."""
    # Plug in x and y coordinates provided
    response = requests.get(URL.format(x, y))
    # Parse JSON with received z-coordinate
    return response.json()['z']
# Initial agent coordinates
START_X = 0
START_Y = 0
# Path traversed by the agent
PATH = []

#### 1. Grid sample the environment
Strategy:
1. Spend 25 explorations into getting 5x5 map representation of the environment.
2. If the relationship is linear, then the environment is deterministic.
3. Otherwise, the gridline would suggest where to look for the target.

In [3]:
def get_grid_points(depth: int, step: float, name: str) -> list[tuple]:
    """Query a rectangular map representation of the environment."""
    res = []
    size = 2 * depth + 1
    # Iterate over rows and columns
    desc = f'Downloading \"{name}\" {size}x{size} sample'
    for y in tqdm(reversed(range(-depth, depth + 1)), total=size, desc=desc):
        for x in range(-depth, depth + 1):
            z = make_api_call(x * step, y * step)
            res.append((x * step, y * step, z))
    return res
# Explore square diagonals from zero value
grid_7_x_7_points = get_grid_points(2, 50, 'initial grid points')

#### 2. Fit linear regression model

In [4]:
# Convert to numpy
grid_7_x_7_arr = np.array(grid_7_x_7_points)
xy = grid_7_x_7_arr[:, :2]
z = grid_7_x_7_arr[:, 2]
# Create a linear regression model and fit it to the grid sample
model = LinearRegression()
model.fit(xy, z)
# The coefficients and intercept of the plane
a, b = model.coef_
c = model.intercept_
print(f"The linear function is: z(x, y) = {a:.2f}x + {b:.2f}y + {c:.2f}")

#### 3.1 Verify linearity of data

In [5]:
def prove_linearity(_model, _xy, _z, threshold_r2=0.8, threshold_mse=10) -> bool:
    """Prove that the dataset relationship is linear."""
    z_pred = _model.predict(_xy)
    # Calculate R-squared
    r2 = r2_score(_z, z_pred)
    # Calculate Mean Squared Error
    mse = mean_squared_error(_z, z_pred)
    # Check if R-squared is above the threshold and MSE is below the threshold
    if r2 >= threshold_r2 and mse <= threshold_mse:
        return True
    else:
        return False
# Prove linearity
is_linear = prove_linearity(model, xy, z)
is_linear

#### 3.2 Solve the deterministic environment knowing the linear function with the brute force

In [6]:
if is_linear:
    # Replicate the function on the server
    linear_function = lambda x, y: a * x + b * y + c
    # Find all coordinates in the deterministic environment
    all_points = []
    for y in reversed(range(-100, 100 + 1)):
        for x in range(-100, 100 + 1):
            z = linear_function(x, y)
            all_points.append((x, y, z))
    # Sort the list of all points by the z-coord value
    all_points_sorted = sorted(all_points, key=lambda x: x[2])
    # Select top 10 values
    PATH = all_points_sorted[-10:]
    pprint(PATH)

#### 3.3 Build a path through the 10 best points using brute force

In [7]:
def calculate_euclidean_distance(p1, p2):
    """Find the distance between two points."""
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def find_hamiltonian_path(points):
    """Find the hamiltonian path between the richest points."""
    n = len(points)
    min_distance = float('inf')
    min_path = None

    for perm in permutations(points):
        total_distance = 0
        for i in range(n - 1):
            total_distance += calculate_euclidean_distance(perm[i], perm[i + 1])

        if total_distance < min_distance:
            min_distance = total_distance
            min_path = perm

    return min_path

if is_linear:
    # Make the list of points
    incorrect_path = [(x, y) for x, y, _ in PATH]
    # Find the most optimal path between points
    hamiltonian_path = find_hamiltonian_path(incorrect_path)
    # Create a DataFrame
    df = pd.DataFrame(hamiltonian_path, columns = ['x', 'y'])
    # Add theoretical and practical values
    df['z'] = df.apply(lambda row: make_api_call(row['x'], row['y']), axis=1)
    df['z_theory'] = df.apply(lambda row: linear_function(row['x'], row['y']), axis=1)
    #  Save path
    df[['x', 'y']].to_csv('explores_01.csv', index=False, header=False)
    print(df.head(10))
    print(f"Sum of all z-values traversed is {df['z'].sum()} (expected={df['z_theory'].sum()})")